In [3]:
import requests
import json
import datetime
import time
from selectolax.parser import HTMLParser
import pandas as pd
import re
import os

In [27]:
df = pd.read_csv("zhihu_urls_chtgpt.csv")

In [28]:
original_questionId = []
def getQuestionId(href):
    res = re.findall(r'https://www.zhihu.com/question/([0-9]+)',href)
    if(len(res)!=0):
        original_questionId.append(res[0])
for url in pd.read_csv("../Addition_zhihu_AI/zhihu_urls.csv"):
    getQuestionId(url)
for url in pd.read_csv("../Addition_zhihu_AI/zhihu_urls_addition.csv")['link-href']:
    getQuestionId(url)

In [29]:
questionId = []
def getQuestionId(href):
    res = re.findall(r'https://www.zhihu.com/question/([0-9]+)',href)
    if(len(res)!=0 and res not in original_questionId):
        questionId.append(res[0])
for url in df['link-href']:
    getQuestionId(url)

In [30]:
questionId = list(set(questionId))
len(set(questionId))

72

In [31]:
questionName = []

for q in questionId:
    for i, n in enumerate(df['link-href']):
        if q in n:
            questionName.append(df['link'][i])

In [32]:
questionName

['微软宣布全线整合 ChatGPT，将带来哪些影响？',
 '如何看待ChatGPT开始收费？专业版每月42美元，这个价格定位可以接受吗？',
 '有没有中国版的chatGPT?',
 'ChatGPT都有些什么好玩的玩法？',
 '有没有ml，nlp方面的大佬说说对chatgpt的看法？',
 'ChatGPT能否帮助人类进行哲学思考和研究？',
 '国内有类似 ChatGPT 能力的模型吗？',
 '可不可以将ChatGPT生成的内容在放在CSDN上？',
 '清华计算机教授称 ChatGPT 对话水平已超过 9 成人类，这有哪些重大意义？',
 '如何评价 OpenAI 推出付费版 ChatGPT Pro?',
 'ChatGPT会淘汰哪些行业和APP？',
 '为什么百度没有做出chatGPT这种产品？',
 '如何让chatgpt假装成我的女友?',
 '微软产品将全线接入 ChatGPT，包括 Bing 以及 Office「全家桶」，将产生哪些影响？',
 '最近被chatGPT和AlphaFold2惊艳到了，产生如下问题，请畅所欲言，谢谢?',
 '全球爆红的 ChatGPT 是如何诞生的？ChatGPT 的出现给商业巨头带来了哪些冲击和变革？',
 '如何评价chatgpt?你感觉它会在哪些方面提高我们的效率？',
 '斯坦福推出新算法严查 ChatGPT 代写论文，你对此怎么看？',
 'Google LaMDA 和 OpenAI ChatGPT 哪个更强大？',
 'OpenAI 的超级对话模型 ChatGPT 会导致程序员大规模失业吗？',
 '国内那么多杰青优青，那么多项目，有些学者一年几十篇顶会论文，怎么做不出chatGPT这种工作？',
 '目前ChatGPT 已应用到论文写作、剧本创作、媒体内容生产，是解放生产力的机会还是被AI支配的开始？',
 '在国内，如何玩一下chatgpt？',
 'ChatGPT 这个项目会开源吗？',
 '使用chatGPT在线对话的时候，比较长的答案每次都说一半就结束了，不完整是为什么？',
 '网传百度计划 3 月推出国产版 ChatGPT ，将会对传统搜索引擎带来什么影响？',
 'ChatGPT概念有哪些股票可以关注？',
 'ChatGPT 的训练集来自哪里？',
 '现在还有没有能用的Chat

In [33]:
questionId

['581818681',
 '580195832',
 '576529182',
 '571619870',
 '580504936',
 '582041292',
 '570713548',
 '572102382',
 '582209155',
 '580301521',
 '582392578',
 '572106826',
 '580758293',
 '582312398',
 '580085374',
 '581583010',
 '572325669',
 '581531522',
 '570342538',
 '570403406',
 '582144133',
 '581304464',
 '570939438',
 '571390218',
 '573381722',
 '581347153',
 '581854293',
 '570431477',
 '578127843',
 '571009463',
 '571422186',
 '571463360',
 '582421050',
 '582391675',
 '581311491',
 '581825806',
 '582079580',
 '582041586',
 '580642633',
 '581872941',
 '580376712',
 '580801093',
 '578268304',
 '571919681',
 '575483456',
 '570788924',
 '570596331',
 '577540847',
 '570520300',
 '575481512',
 '570406559',
 '581802648',
 '578492084',
 '571445961',
 '577494357',
 '570951234',
 '582276511',
 '571087237',
 '581464703',
 '574591812',
 '571387160',
 '581307773',
 '570819179',
 '55445739',
 '581952618',
 '582326598',
 '570990253',
 '571820396',
 '572265386',
 '579037511',
 '581821187',
 '58180

In [12]:
def download_answer(question_id, start):
#    基于问题id获取单个问题下所有回答数据信息
    print(f"问题{question_id},第{start}条。")
    url = "https://www.zhihu.com/api/v4//questions/{}/answers".format(question_id)
    print(url)
    headers = {'User-Agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Mobile Safari/537.36',
               'referer':'https://www.zhihu.com/question/{}'.format(question_id),
              } 
    
    para = {
    'include':'data[*].is_normal,admin_closed_comment, \
    reward_info,is_collapsed,annotation_action,annotation_detail,\
    collapse_reason,is_sticky,collapsed_by,suggest_edit,comment_count,\
    can_comment,content,editable_content,attachment,voteup_count,\
    reshipment_settings,comment_permission,created_time,\
    updated_time,review_info,relevant_info,question,excerpt,is_labeled,\
    paid_info,paid_info_content,relationship.is_authorized,is_author,voting,is_thanked,\
    is_nothelp,is_recognized;data[*].mark_infos[*].url;data[*].author.follower_count,\
    vip_info,badge[*].topics;data[*].settings.table_of_content.enabled',
    'offset':start,
    'limit':20,
    'sort_by':'default',
    'platform':'desktop'
    }
    
    global data   

    response = requests.get(url,params=para,headers = headers)
    response.encoding = response.apparent_encoding
    if response.status_code == 200:
        res = json.loads(response.text)
        if res['data']:
            for answer in res['data']:
                author = answer['author']['name']
                fans = answer['author']['follower_count'] 
                content = HTMLParser(answer['content']).text()
                #content = answer['content']
                created_time = datetime.datetime.fromtimestamp(answer['created_time'])
                updated_time = datetime.datetime.fromtimestamp(answer['updated_time'])
                comment = answer['comment_count']
                voteup = answer['voteup_count']
                link = answer['url']
                row = {
                    'author':[author],
                    'fans_count':[fans],
                    'content':[content],
                    'created_time':[created_time],
                    'updated_time':[updated_time],
                    'comment_count':[comment],
                    'voteup_count':[voteup],
                    'url':[link]
                }
                data = data.append(pd.DataFrame(row),ignore_index=True)
            if len(res['data'])==20: 
                download_answer(question_id,start+20)
                time.sleep(1)
        else:
            return None
    else:
        return None

In [13]:
data = pd.DataFrame(columns=('author','fans_count','content','created_time','updated_time','comment_count','voteup_count','url'))

for i,id in enumerate(questionId):
    try:
        download_answer(id,0)
        print('#'*20+str(i)+f'/{len(questionId)}'+'#'*20)
    except: 
        continue

问题572106826,第0条。
https://www.zhihu.com/api/v4//questions/572106826/answers


/var/folders/29/27cgqwrx7_18wcd66rpffzlc0000gn/T/ipykernel_74272/1512429315.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(pd.DataFrame(row),ignore_index=True)


问题572106826,第20条。
https://www.zhihu.com/api/v4//questions/572106826/answers
####################0/72####################
问题578492084,第0条。
https://www.zhihu.com/api/v4//questions/578492084/answers
问题578492084,第20条。
https://www.zhihu.com/api/v4//questions/578492084/answers
问题578492084,第40条。
https://www.zhihu.com/api/v4//questions/578492084/answers
####################1/72####################
问题578127843,第0条。
https://www.zhihu.com/api/v4//questions/578127843/answers
####################2/72####################
问题581311491,第0条。
https://www.zhihu.com/api/v4//questions/581311491/answers
问题581311491,第20条。
https://www.zhihu.com/api/v4//questions/581311491/answers
问题581311491,第40条。
https://www.zhihu.com/api/v4//questions/581311491/answers
问题581311491,第60条。
https://www.zhihu.com/api/v4//questions/581311491/answers
问题581311491,第80条。
https://www.zhihu.com/api/v4//questions/581311491/answers
####################3/72####################
问题570939438,第0条。
https://www.zhihu.com/api/v4//questions/570939

问题570520300,第220条。
https://www.zhihu.com/api/v4//questions/570520300/answers
####################28/72####################
问题581806122,第0条。
https://www.zhihu.com/api/v4//questions/581806122/answers
####################29/72####################
问题581583010,第0条。
https://www.zhihu.com/api/v4//questions/581583010/answers
问题581583010,第20条。
https://www.zhihu.com/api/v4//questions/581583010/answers
问题581583010,第40条。
https://www.zhihu.com/api/v4//questions/581583010/answers
问题581583010,第60条。
https://www.zhihu.com/api/v4//questions/581583010/answers
问题581583010,第80条。
https://www.zhihu.com/api/v4//questions/581583010/answers
####################30/72####################
问题570951234,第0条。
https://www.zhihu.com/api/v4//questions/570951234/answers
问题570951234,第20条。
https://www.zhihu.com/api/v4//questions/570951234/answers
####################31/72####################
问题581952618,第0条。
https://www.zhihu.com/api/v4//questions/581952618/answers
####################32/72####################
问题570342538,第

问题570403406,第120条。
https://www.zhihu.com/api/v4//questions/570403406/answers
问题570403406,第140条。
https://www.zhihu.com/api/v4//questions/570403406/answers
问题570403406,第160条。
https://www.zhihu.com/api/v4//questions/570403406/answers
问题570403406,第180条。
https://www.zhihu.com/api/v4//questions/570403406/answers
问题570403406,第200条。
https://www.zhihu.com/api/v4//questions/570403406/answers
####################64/72####################
问题580085374,第0条。
https://www.zhihu.com/api/v4//questions/580085374/answers
####################65/72####################
问题581307773,第0条。
https://www.zhihu.com/api/v4//questions/581307773/answers
问题581307773,第20条。
https://www.zhihu.com/api/v4//questions/581307773/answers
####################66/72####################
问题570990253,第0条。
https://www.zhihu.com/api/v4//questions/570990253/answers
####################67/72####################
问题572325669,第0条。
https://www.zhihu.com/api/v4//questions/572325669/answers
####################68/72####################
问题5713902

In [14]:
data.to_csv('data_zhihu_addition_ChatGPT.csv',index=False, encoding='utf-8-sig')